# The Context Manager aka the Measurement Object

This notebook shows some ways of performing different measurements using 
QCoDeS parameters and the new DataSet accessed via a context manager.


In [1]:
%matplotlib notebook
import numpy.random as rd
import matplotlib.pyplot as plt
from functools import partial
import numpy as np

from time import sleep, monotonic

import qcodes as qc
from qcodes import Station, load_or_create_experiment, \
    initialise_database, Measurement, load_by_run_spec, load_by_guid
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.dataset.plotting import plot_dataset
import time
qc.logger.start_all_logging()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\jenielse\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


In [2]:
# a generator to simulate a physical signal, in this case an exponentially
# decaying signal

def exponential_decay(a: float, b: float):
    """
    Yields a*exp(-b*x) where x is put in 
    """
    x = 0
    while True:
        x = yield
        yield a*np.exp(-b*x) + 0.02*a*np.random.randn()

In [3]:
# preparatory mocking of physical setup

dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
dmm = DummyInstrument('dmm', gates=['v1', 'v2'])

station = qc.Station(dmm, dac)

In [4]:
# and then a bit of "wiring" to make the dmm "measure"
# the exponential decay

ed = exponential_decay(5, 0.2)
next(ed)

def customgetter(dac):
    val = ed.send(dac.ch1())
    next(ed)
    return val
    
dmm.v1.get = partial(customgetter, dac)

In [5]:
# now make some silly set-up and tear-down actions

def veryfirst():
    print('Starting the measurement')

def numbertwo(inst1, inst2):
    print('Doing stuff with the following two instruments: {}, {}'.format(inst1, inst2))
    
def thelast():
    print('End of experiment')

**Database and experiments may be missing**

If this is the first time you create a dataset, the underlying database file has
most likely not been created. The following cell creates the database file. Please
refer to documentation on `The Experiment Container` for details.

Furthermore, datasets are associated to an experiment. By default the run
is appended to the latest existing experiments. If no experiment has been created,
we must create one. We do that by calling `load_or_create_experiment`

Here we explicitly pass the loaded or created experiment to the `Measurement` to ensure that we are always
useing the `dataset_context_manager` experiment throughout this tutorial.

In [6]:
initialise_database()
exp = load_or_create_experiment(experiment_name='dataset_context_manager',
                          sample_name="no sample")

In [7]:
exp

dataset_context_manager#no sample#3@C:\Users\jenielse\experiments.db
--------------------------------------------------------------------
10-results-1-dac_ch1,dmm_v1-10
11-results-2-dac_ch1,dmm_v1-32
12-results-3-dac_ch1,dmm_v1-98
13-results-4-dac_ch1,dac_ch2,dmm_v1-40000
14-results-5-dac_ch1,dac_ch2,dmm_v1-44100
15-results-6-dac_ch1,dac_ch2,dmm_v1-20368
16-results-7-dac_ch1,dac_ch2,dmm_v1-5000
17-results-8-dac_ch1,dac_ch2,dmm_v1-77
18-results-9-dac_ch1,dmm_v1-10
19-results-10-dac_ch1,dmm_v1-10
20-results-11-MyDummy_ChanA_this_setpoint,MyDummy_ChanA_that_setpoint,this,that-30
49-results-12-dac_ch1,dmm_v1-10
50-results-13-dac_ch1,dmm_v1-10
51-results-14-dac_ch1,dmm_v1-0
52-results-15-dac_ch1,dmm_v1-10
53-results-16-dac_ch1,dmm_v1-10
54-results-17-dac_ch1,dmm_v1-0

In [11]:
# And then run an experiment

meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)  # register the first independent parameter
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))  # now register the dependent oone
meas.add_before_run(veryfirst, ())  # add a set-up action
meas.add_before_run(numbertwo, (dmm, dac))  # add another set-up action
meas.add_after_run(thelast, ())  # add a tear-down action

meas.write_period = 2


with meas.run() as datasaver:
    time.sleep(20)
    # While sleeping here start loader. From load_cached_notebook.ipynb
    print("done sleeping")
    for set_v in np.linspace(0, 25, 100):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v),
                             (dmm.v1, get_v))
        # flush so this always works
        datasaver.flush_data_to_database()
        time.sleep(0.1)

    
    dataset = datasaver.dataset  # convenient to have for plotting


Starting the measurement
Doing stuff with the following two instruments: <DummyInstrument: dmm>, <DummyInstrument: dac>
Starting experimental run with id: 58
done sleeping
End of experiment
